# Handling VERITAS spectra

After defining a data file format for VERITAS spectra -- https://github.com/chbrandt/veritas/blob/master/data_formatting-v2.rst -- we'll go here handle those files to see whether they fit our needs.

The files we are going to handle here are all from the same object: Mkn 421.
When an object is observed multiple times, under different activity levels, the VERITAS collaboration has chosen to split the corresponding *spectra* according to the object's activity.

Since an object can have multiple *spectra* associated with it, there must be a way to filter for the data points
of interest -- understanding the user will fine-tune the data according to its science case.
Straightforward solution for breaking down this degeneracy (same object for multiple tables) is to use the observations `epoch`; since no two events can occur at the same time and location, the moment of observation must be unique for each data file (i.e, *spectra*).
(To be precise, since the VHE astrophysics is a place where photon are somewhat rare, in a quiescent period it is necessary a long time to integrate the amount of light necessary to build up an *spectra*, if a *flare* happens during this period, the *low emission* *spectra* will contain the period of the flare; whereas the *high emission spectra* of the same object will correspond to the *flare* period only.)

When handlig the *spectra* datasets, the goal is to have a table with `energy`, `flux` and `epoch` so that plotting and modelling can be done.
Here, that is the place we have to arrive: (*i*) one plot `energy` *vs* `flux` and (*ii*) one plot `epoch` *vs* `flux`.

## Testing case: Mkn421

Seven data files with *spectra* from Mkn421 are available to build those SED plots.
The files are in ECSV format as proposed in https://github.com/chbrandt/veritas/blob/master/data_formatting-v2.rst.

In [4]:
%ls

handling_mkn421.ipynb          Mkn421_VERITAS_2008_low.csv
Mkn421_VERITAS_2008_highA.csv  Mkn421_VERITAS_2008_mid.csv
Mkn421_VERITAS_2008_highB.csv  Mkn421_VERITAS_2008_veryhigh.csv
Mkn421_VERITAS_2008_highC.csv  Mkn421_VERITAS_2008_verylow.csv


In [5]:
%cat 'Mkn421_VERITAS_2008_highA.csv'

# %ECSV 0.9
# ---
# meta: !!omap
# - object: Mrk 421
#
# - description:
#    Spectral points for multiwavelength campaign;
#    Observations taken between 2008 January 01 and 2008 June 05;
#    Flux sensitivity 0.8e-10 < flux(E>1TeV) < 1.1e-10
#
# - mjd:
#    start: 54502.46971
#    end: 54622.18955
#
# - article:
#    label: Ap.J. 738, 25 (2011)
#    url: http://iopscience.iop.org/0004-637X/738/1/25/
#    arxiv: http://arxiv.org/abs/1106.1210
#    ads: http://adsabs.harvard.edu/abs/2011ApJ...738...25A
#
# - comments:
#    - Name=Mrk421_2008_highA
#    - z=0.031
#    - LiveTime(h)=1.4
#    - significance=73.0
#
# - SED_TYPE: diff_flux_points
#
# datatype:
# - name: e_ref
#   unit: TeV
#   datatype: float64
# - name: dnde
#   unit: ph / (m2 TeV s)
#   datatype: float64
# - name: dnde_errn
#   unit: ph / (m2 TeV s)
#   datatype: float64
# - name: dnde_errp
#   unit: ph / (m2 TeV s)
#   datatype: float64
#
e_ref dnde       dnde_errn  dnde_errp
0.275 1.702E-005 3.295E-006 3.295E-006
0.340 

The format -- ECSV -- has been chosen for it is human readable and metadata rich.
The library we have to use is Astropy.

In [6]:
from astropy.table import Table
tt = Table.read('Mkn421_VERITAS_2008_highA.csv', format='ascii.ecsv')

In [10]:
tt

e_ref,dnde,dnde_errn,dnde_errp
TeV,ph / (m2 s TeV),ph / (m2 s TeV),ph / (m2 s TeV)
float64,float64,float64,float64
0.275,1.702e-05,3.295e-06,3.295e-06
0.34,1.289e-05,1.106e-06,1.106e-06
0.42,8.821e-06,6.072e-07,6.072e-07
0.519,5.777e-06,3.697e-07,3.697e-07
0.642,3.509e-06,2.351e-07,2.351e-07
0.793,2.151e-06,1.525e-07,1.525e-07
0.98,1.302e-06,1.024e-07,1.024e-07
1.212,6.273e-07,6.117e-08,6.117e-08


In [9]:
tt.meta

OrderedDict([('object', 'Mrk 421'),
             ('description',
              'Spectral points for multiwavelength campaign; Observations taken between 2008 January 01 and 2008 June 05; Flux sensitivity 0.8e-10 < flux(E>1TeV) < 1.1e-10'),
             ('mjd', {'end': 54622.18955, 'start': 54502.46971}),
             ('article',
              {'ads': 'http://adsabs.harvard.edu/abs/2011ApJ...738...25A',
               'arxiv': 'http://arxiv.org/abs/1106.1210',
               'label': 'Ap.J. 738, 25 (2011)',
               'url': 'http://iopscience.iop.org/0004-637X/738/1/25/'}),
             ('comments',
              ['Name=Mrk421_2008_highA',
               'z=0.031',
               'LiveTime(h)=1.4',
               'significance=73.0']),
             ('SED_TYPE', 'diff_flux_points')])